In [1]:
from google.colab import auth 
auth.authenticate_user()  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
## torch와 torchtext 버전이 맞지 않아 생기는 충돌을 막기 위해 
## 버전을 맞춰 설치 


%cd  /usr/local/lib/python3.7/dist-packages/
%rm -rf torch
!pip install torch==1.7.0
!pip install torchtext==0.8.1
%cd /content/gdrive/MyDrive/nlg/machine_translation/nmt

/usr/local/lib/python3.7/dist-packages
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 7.0MB 7.1MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
/content/gdrive/MyDrive/nlg/machine_translation/nmt


In [4]:
%cd /content/gdrive/MyDrive/nlg/machine_translation/nmt

/content/gdrive/MyDrive/nlg/machine_translation/nmt


In [5]:
import os
## this is to prevent RuntimeError: CUDA error: device-side assert triggered
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

In [6]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence 
from torch.nn.utils.rnn import pad_packed_sequence as unpack_padded_sequence
import torchtext
from torchtext.data import Field, Dataset, Example, interleave_keys, BucketIterator
import torch.optim as optim 
import torch 
import torch.nn as nn 
from torch.cuda.amp import GradScaler, autocast
import numpy as np
from torch.nn import utils as tc_utils
import random

#random.seed(1)
#np.random.seed(1)
#np.random.RandomState(1)
#torch.manual_seed(1)
#torch.cuda.manual_seed(1)
#torch.backends.cudnn.enabled=False
#torch.backends.cudnn.benchmark = False
#torch.backends.cudnn.deterministic=True

In [8]:
## Hyper Parameter for Data 
train_src_fn = '../data/corpus.shuf.train.tok.bpe.en'
train_tgt_fn = '../data/corpus.shuf.train.tok.bpe.ko'
valid_src_fn = '../data/corpus.shuf.valid.tok.bpe.en'
valid_tgt_fn = '../data/corpus.shuf.valid.tok.bpe.ko'
max_len = 100
init_token = '<BOS>'
eos_token= '<EOS>'
batch_size= 48
device = 0
max_vocab = 999999999999

In [8]:
## this is why pad_id = 1, init_token_id=2, eos_token_id = 3
 
unk_token = '<unk>'
pad_token = '<pad>'
init_token = '<BOS>'
eos_token =  '<EOS>'

## this code from pytorch document
from collections import OrderedDict
list(OrderedDict.fromkeys(
            tok for tok in [unk_token, pad_token, init_token,
                            eos_token]
            if tok is not None))

['<unk>', '<pad>', '<BOS>', '<EOS>']

In [7]:
## Neural Machine Translation Dataset 



class TranslationDataset(Dataset) : 
  def __init__(self, src_fn, tgt_fn, fields, max_len) :
    src_field = ('src', fields[0])
    tgt_field = ('tgt', fields[1]) 

    examples = list()

    with open(src_fn, encoding='utf-8') as src, open(tgt_fn, encoding='utf-8') as tgt : 
      for cur_src, cur_tgt in zip(src, tgt) : 
        cur_src, cur_tgt = cur_src.strip(), cur_tgt.strip()
        if max_len and (max_len < max(len(cur_src.split()), len(cur_src.split()))) : continue
        examples.append(Example.fromlist([cur_src, cur_tgt], [src_field, tgt_field]))

    super().__init__(examples, [src_field, tgt_field])

  @staticmethod
  def sort_key(ex) : 
    return interleave_keys(len(ex.src), len(ex.tgt))



class DataLoader(object) : 
  PAD_ID = 1
  INIT_TOKEN_ID = 2
  EOS_TOKEN_ID = 3

  def __init__(self, src_fn=None, tgt_fn=None, max_len=255, init_token='<BOS>', eos_token='<EOS>', batch_size = 32, device=-1, shuffle=None, max_vocab =99999999999) : 
    super(DataLoader, self).__init__()
    self.src = Field(sequential=True,use_vocab=True, batch_first=True,
                    include_lengths=True,fix_length=None,
                    init_token=None,eos_token=None)
    self.tgt = Field(sequential=True,use_vocab=True, batch_first=True,
                    include_lengths=True,fix_length=None,
                    init_token=init_token, eos_token=eos_token)
    
    if src_fn != None and tgt_fn != None :
      transl_dataset = TranslationDataset(src_fn, tgt_fn, (self.src, self.tgt), max_len)

      self.data_iter = BucketIterator(transl_dataset, batch_size=batch_size, 
                                      device = 'cuda:%d' % device if device >= 0 else 'cpu',
                                      sort_key = lambda x: len(x.tgt) +(max_len*len(x.src)),
                                      shuffle = shuffle,
                                       sort_within_batch=True,
                       )
      
      self.src.build_vocab(transl_dataset, max_size=max_vocab)
      self.tgt.build_vocab(transl_dataset, max_size=max_vocab)

      
  def set_vocab_for_reproduction(self, src_vocab, tgt_vocab) : 
    self.src.vocab = src_vocab 
    self.tgt.vocab = tgt_vocab 

In [10]:
train_dl =  DataLoader(train_src_fn, train_tgt_fn, max_len, init_token, eos_token, batch_size,  -1, True, max_vocab)   ## device = -1 because gpu memory leakage -> batchwise loading 
valid_dl =  DataLoader(valid_src_fn, valid_tgt_fn, max_len, init_token, eos_token, batch_size,  -1, False,max_vocab)   ## device = -1 because gpu memory leakage -> batchwise loading 

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information

In [11]:
valid_dl.set_vocab_for_reproduction(train_dl.src.vocab, train_dl.tgt.vocab)

In [12]:
num_epochs = 30

input_size = len(train_dl.src.vocab)
output_size = len(train_dl.tgt.vocab)

embed_size = 512
hidden_size = 768
num_layer = 4
enc_bidirectional = True
drop_out = 0.2
max_clip_norm = 1e+8


beam_size = 5
lp_alpha = 0.6
lp_length = 5
cp_beta = 0.4
cp_min_val = 0.4

In [13]:
learning_rate =0.001
grad_update_period = 4

In [14]:
## Attention 기반 Neural Machine Translator의 구성 
## Encoder + Decoder + Generator
## 그 외: 예측을 위해 BeamSearch를 사용, masking

In [8]:
class Attention(nn.Module) : 
  def __init__(self, hidden_size) :
    super(Attention,self).__init__()

    self.make_query = nn.Linear(hidden_size, hidden_size, bias=False)
    self.softmax = nn.Softmax(dim=-1) 

  def forward(self, enc_hs, cur_dec_h, mask=None) :
    ## |enc_hs| = (batch_size, enc_length, hidden_size)
    ## |cur_dec_h| = (batch_size, 1, hidden_size)
    ## |mask| = (batch_size, length)

    query = self.make_query(cur_dec_h)
    ## |query| = (batch_size, 1, hidden_size)
    
    attn_weight = torch.bmm(query, enc_hs.transpose(1,2))
    ## |attn_weights| = (batch_size, 1, enc_length)

    if mask is not None :
      attn_weight.masked_fill_(mask.unsqueeze(1), -float('inf'))

    ## normalize and masking 
    attn_weight = self.softmax(attn_weight)
    ## |attn_weights| = (batch_size, 1, enc_length)

    context_vector = torch.bmm(attn_weight, enc_hs)
    ## |attn_weights} = (batch_size, 1, hidden_size)

    return context_vector, attn_weight 

In [9]:
class LSTMEncoder(nn.Module) : 

  def __init__(self, embed_size, hidden_size, batch_first=True, num_layer=4, bidirectional=False, dropout=0.2) :
    super(LSTMEncoder,self).__init__() 

    ## hidden size를 hidden_size//2로 하는 이유 -> LSTM을 bidirectional로 하고 이를 concat하여
    ## LSTMEncoder의 최종 hidden size가 hidden_size가 되게 하기 위함 -> 이를 위해서는 hidden 
    ## size가 짝수가 되어야 함 
    self.encoder = nn.LSTM(input_size=embed_size, hidden_size=int(hidden_size/2), 
                           num_layers=num_layer, batch_first=batch_first, 
                           bidirectional=bidirectional, dropout=dropout)

  def forward(self, emb_vec) : 
    ## 1. emb_vec = (emb_vec_data, emb_vec_length)의 튜플인 경우 
    ##   |emb_vec_data| = (batch_size, enc_length, embed_size)
    ##   |emb_vec_length| = (batch_size, )
    ## 2. emb_vec = emb_vec_data 
    ##   |emb_vec_data| = (batch_size, enc_length, embed_size)

    if isinstance(emb_vec, tuple) :
      ## emb_vec_data와 emb_vec_length는 mini batch 내에서 pad를 제외한 토큰의 개수를 기준으로
      ## 내림차순으로 정렬되어 있음. 이는 효율성 뿐만 아니라 pad를 계산하지 않아 정확도를 높이
      ## 기 위한 목적도 존재 
      ## 출처 : https://simonjisu.github.io/nlp/2018/07/05/packedsequence.html
      inp, emb_vec_length = emb_vec
      inp = pack_padded_sequence(inp, emb_vec_length.tolist(), batch_first=True)
    else : 
      inp = emb_vec
    
    h_enc, hc_per_layer  = self.encoder(inp)
    ## |h_enc| = (batch_size, hidden_size)
    ## |hc_per_layer[0]| = (2*layer_num, batch_size, (hidden_size//2)) ([0] for hidden state, [1] for cell state)

    if isinstance(emb_vec, tuple) : 
      h_enc, _ = unpack_padded_sequence(h_enc, batch_first=True)

    return h_enc, hc_per_layer


In [10]:
class LSTMDecoder(nn.Module) : 
  def __init__(self, embed_size, hidden_size, batch_first=True, num_layer=4, dropout=0.2) :
    super(LSTMDecoder,self).__init__()

    self.embed_size = embed_size 
    self.hidden_size = hidden_size

    ## Decoder는 Autoregressive한 속성을 가지기 때문에 하나의 Decoder가  
    ## 양 방향을 모두 고려할 수 없다. 
    self.decoder = nn.LSTM(
        input_size=embed_size+hidden_size,
        hidden_size=hidden_size,
        num_layers=num_layer,
        batch_first = batch_first, 
        dropout = dropout,
        bidirectional=False
    )

   
  def forward(self, embed_vec, prev_dec_out, prev_hc) :
   ## |emb_vec| = (batch_size, 1, embed_size) -> 학습 과정에서는 실제 단어, 예측 과정에서는 이전 timestep에서의 예측값 
   ## |prev_hc| 
   ##   |prev_h| = (num_layers, batch_size, hidden_size) -> 이전 타임 스텝의 hidden state
   ##   |prev_c| = (num_layers, batch_size, hidden_size) -> 이전 타임 스텝의 cell state(LSTM 기반이기 때문)
   ## |prev_dec_out| = (batch_size, 1, hidden_size) -> input feeding을 위한 것 

    ## 맨 처음에는 이전 timestep이 없으므로 이전 timestep의 예측값도 없다. 이 경우 이전 time 
    ## step의 output을 null vector로 해준다. 이 때 기존의 tensor와 같은 device에 있으면서 
    ## 같은 디바이스의 tensor로 만들기 위해 new_zeros()를 사용한다. 
    ## 또한 beam search 과정에서는 input feeding을 사용하지 않기 때문에 이러한 경우에도 사용한다.
    batch_size = embed_vec.size(0)
    if prev_dec_out is None : 
      prev_dec_out = embed_vec.new(batch_size, 1, self.hidden_size).zero_()
    
    ## input feeding -> Teacher Forcing을 사용하는 경우 학습 과정과 예측 과정에서 발생하는 
    ##                  모델에게 주어지는 정보의 차이를 최소화하기 위한 방법  
    input_vec = torch.cat([embed_vec, prev_dec_out], dim=-1)
    ## |input_vec| = (batch_size, 1, embed_size + hidden_size)


    h_dec, hc_per_layer = self.decoder(input_vec, prev_hc)
    ## |h_dec| = (batch_size, 1, 2*(hidden_size//2))
    ## |hc_per_layer[0]| = |hc_per_layer[1]| = (num_layers, batch_size, hidden_size)

    return h_dec, hc_per_layer



In [11]:
## Generator는 Decoder의 Output인 Hidden State와 Attention의 Output인 Context Vector를
## 입력 받아 조건부 확률의 결과인 Softmax Layer의 Output을 리턴

class Generator(nn.Module) :
  def __init__(self,hidden_size, output_size) :
    super(Generator, self).__init__()

    self.transform = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=-1)

  def forward(self, h_context) :
    # |dec_output| = (batch_size, length, hidden_size)
    
    transformed_vec = self.transform(h_context)
    # |transformed_vec| = (batch_size, length, out_size)
    
    output = self.softmax(transformed_vec)
    # |output| = (batch_size, length, out_size)
    
    return output

In [12]:
class LengthPenalty(object) :
  def __init__(self, alpha, beta) :
    super(LengthPenalty,self).__init__()
    self.alpha = alpha 
    self.beta = beta 

  def __call__(self, length) :
    ## |length| = (batch_size, 1, 1) 
    return ((1+self.beta) / (1+length))**self.alpha  

In [13]:
class ConvergencePenalty() : 
  def __init__(self, beta, min_val=1) :
    super(ConvergencePenalty,self).__init__()
    self.beta = beta 
    self.min_val = min_val 

  def __call__(self, attn_weights) : 
    ## |attn_weights| = (batch_size, enc_length, dec_length)
    
    log_attn_w = torch.log(attn_weights)
    ## |log_attn_w| = (batch_size, enc_length, dec_length)

    sum_attn_w = torch.sum(log_attn_w, dim=-1)
    ## |sum_attn_w| = (batch_size, enc_length, 1)

    clipped_attn_w = torch.clip(sum_attn_w, max=self.min_val)
    ## |clipped_attn_w| = (batch_size, enc_length, 1)

    coverage_penalty = self.beta * torch.sum(clipped_attn_w, dim=-1)
    ## |covergae_penalty| = (batch_size, 1)

    return coverage_penalty



In [14]:
## Beam Size 어떻게 테스트할지 생각해야 함 -> 일단 decoder 코드 부터 작성하는게 맞을 듯 -> 어차피 테스트 코드나 decoder 코드나 같을 거기 때문 



class BeamSearch(nn.Module) : 
  def __init__(self, beam_size, lp_alpha, lp_length, cp_beta, cp_min_val) : 
    super(BeamSearch, self).__init__()
    self.beam_size = beam_size
    
    self.path_probs =  None #torch.Tensor.new_zeros(batch_size, 1, 1).to(device)
    self.path = []

    self.length_penalty = LengthPenalty(lp_alpha, lp_length)
    self.convergence_penalty = ConvergencePenalty(cp_beta, cp_min_val)

  def forward(self, cur_probs, prev_hc, ctx_refl_dec, mask=None) :
    ## |cur_probs| = (batch_size, 1, output_size) or (beam_size*batch_size, 1, ,output_size)
    ## |prev_hc| 
    ##    |prev_h| = (num_layers, batch_size, hidden_size) or (num_layers, beam_size*batch_size, hidden_size)
    ##    |prev_c| = (num_layers, batch_size, hidden_size) or (num_layers, beam_size*batch_size, hidden_size)
    ## |ctx_refl_dec| = (batch_size,1, hidden_size) or (beam_size*batch_size, 1, hidden_size)
    ## |mask| = (batch_size, 1, hidden_size) or (beam_size*batch_size,)

    cur_path_probs = self.calc_cur_path_probs(cur_probs, mask, self.path_probs==None)
    ## |cur_path_probs| = (batch_size, 1, beam_size*output_size)

    prev_path_id, next_inp = self.select_path(path_probs)
    ## |prev_path_id| = |next_inp| = (beam_size*batch_size, 1, 1)

    next_inp = self.construct_path(prev_path_id,next_inp)
    ## |next_inp| = (beam_size*batch_size, 1, 1)

    
    next_hc = self.prepare_next_step(prev_hc, ctx_refl_dec, prev_path_id)
    ## |next_hc|
    ##    |next_h| = |next_c| = (num_layers, beam_size*batch_size, hidden_size)

    return next_inp, next_hc

    
  def calc_cur_path_probs(self, cur_probs, mask=None, swollen=False) :
    ## |cur_probs| = (batch_size, 1, output_size) or (beam_size*batch_size,1, output_size)
    ## |mask| = (batch_size,) or (beam_size*batch_size,)

    output_size = cur_probs.size(-1)

    ## 끝난 instance의 현재 확률값을 1로 만들어 주기 
    if mask != None :
      mask = torch.tile(mask, cur_probs.size())
      ## |mask| = (batch_size, 1, output_size) or (beam_size*batch_size, 1, output_size) 
      
      cur_probs.masked_fill_(mask,1.0)
      ## |cur_probs| = (beam_size*batch_size, 1, output_size)
    

    ##  현재 확률과 path에 대한 cumulative log prob 구하기
    cur_path_probs = torch.tile(self.path_probs, (1,1,output_size))
    ## |cur_path_probs| = (batch_size, 1, output_size) or (beam_size*batch_size, 1, output_size)

    cur_path_probs = cur_path_probs +  torch.log(cur_probs)
    ## |cur_path_probs| = (batch_size, 1, output_size) or (beam_size*batch_size, 1, output_size)

    if swollen == True : 
      cur_path_probs = torch.tile(cur_path_probs, (self.beam_size, 1, 1)).to(device)
      ## |cur_path_probs| = (beam_size*batch_size, 1, output_size)

    cur_path_probs = self.merge_equal_instance(cur_path_probs)
    ## |cur_path_probs| = (batch_size, 1, beam_size*output_size)
    
    return cur_path_probs 

  def merge_equal_instance(self, tensor) : 
    ## |tensor| = (beam_size*batch_size, 1, output_size) or (beam_size*batch_size, 1, 1)
    batch_size = tensor.size(0) // self.beam_size

    split = torch.split(tensor,batch_size, dim=0)
    merged = torch.cat(split, dim=-1)
    ## |merged| = (batch_size, 1, output_size*beam_size) or (batch_size, 1, beam_size)

    return merged 

  def split_equal_instance(self, tensor) :
    ## |tensor| = (batch_size, 1, output_size*beam_size) or (batch_size, 1, beam_size)
    
    split_size = tensor.size(-1) // self.beam_size

    split = torch.split(tensor, split_size, dim=-1)
    split = torch.cat(split, dim=0)
    ## |split| = (beam_size*batch_size, 1, output_size) or (beam_size*batch_size, 1, 1)

    return split 
  
  def select_path(self, path_probs) :
    ## |path_probs| = (batch_size , 1, beam_size*output_size)

    selected_path_probs, selected_token_id = torch.topk(path_probs, self.beam_size, dim=-1)
    ## |selected_path_probs| = |selected_path_id| = (batch_size, 1, beam_size)

    ## for next time step
    self.path_probs = self.split_equal_instance(selected_path_probs)
    ## |selected_path_probs| = (beam_size*batch_size, 1, 1)

    ## this is for selecting next hidden/cell states and input feeding  
    prev_path_id = torch.round(selected_token_id/output_size).type(torch.long)
    prev_path_id = self.split_equal_instance(prev_path)
    ## |prev_path_id| = (beam_size*batch_size, 1, 1)

    next_inp = selected_token_id.type(torch.long) % output_size
    ## |next_inp| = (batch_size, 1, beam_size)

    next_inp = self.split_equal_instance(next_inp)
    ## |next_inp| = (beam_size * batch_size, 1, 1)

    return prev_path_id, next_inp

  ## must call this method after select_path 
  def prepare_next_step(self, prev_hc, ctx_refl_dec, prev_path_id) : 
    ## |ctx_refl_dec| = (beam_size*batch_size,1,hidden_size) or (batch_size, 1, hidden_size)
    ## |prev_path_id| = (beam_size*batch_size,1,1)

    batch_size = prev_path_id.size(0) // self.beam_size    
    
    prev_h, prev_c = prev_hc
    ## |prev_hc| 
    ##    |prev_h| = (num_layers, beam_size*batch_size, hidden_size) or (num_layers, batch_size, hidden_size)
    ##    |prev_c| = (num_layers, beam_size*batch_size, hidden_size) or (num_layers, batch_size, hidden_size)  

    if prev_h.size(1) == batch_size :
      prev_h = torch.tile(prev_h,(1,self.beam_size,1))
      prev_c = torch.tile(prev_c,(1,self.beam_size,1))
      ctx_refl_dec = torch.tile(ctx_refl_dec,(1,self.beam_size,1))
      ## |prev_h| = |prev_c| = |ctx_refl_dec| = (beam_size*batch_size, 1, hidden_size)

    prev_path_id = torch.cat([ i%batch_size+batch_size*path_id for i, path_id in enumerate(prev_path_id)], dim=0)
    ## |prev_path_id| = (beam_size*batch_size, )


    prev_h = prev_h.transpose(0,1)
    prev_c = prev_c.transpose(0,1)
    ## |prev_h| = (beam_size*batch_size, num_layers, hidden_size)

    next_h = prev_h[prev_path_id].transpose(0,1).contiguous()
    next_c = prev_c[prev_path_id].transpose(0,1).contiguous()
    ## |next_h|=|next_c|=(num_layers, beam_size*batch_size, hidden_size)

    return (next_h, next_c)

  ## must call this method after select_path
  def construct_path(self, prev_path_id, next_inp) :
    ## |next_inp| = (beam_size*batch_size, 1, 1) 
    ## |prev_path_id| = (beam_size*batch_size, 1, 1)

    batch_size = prev_path_id.size(0) // self.beam_size
    prev_path_id = torch.cat([i%batch_size + path_id*batch_size for i, path_id in enumerate(prev_path_id)], dim=0)
    ## |prev_path_id| = (beam_size*batch_size)

    if len(self.path) > 0 :
      self.path[-1] = self.path[-1][prev_path_id]
      ## |path[-1]| = (beam_size*batch_size, 1, 1)

    self.path.append(next_inp)
    ## |path| = (cur_length,)
    ##    |path[i]| = (beam_size*batch_size, 1, 1)

    return next_inp

  ## after all time steps
  def make_mask(self, length) :
     
    min_len = torch.min(length)

    ## this is for calc path probability
    ## In this case, the type of length element is boolean type
    if min_len == 0 :
      mask = (length == True)
      ## |length| = (batch_size,) or (beam_size*batch_size,)
      ## |mask| = (batch_size) or (beam_size*batch_size)

    ## this is for cutting off selected path for each instance
    ## In this case, the type of length elements is integer 
    else :
      max_len = torch.max(length)
      idx = torch.arange(max_len).unsqueeze(0).expand((length.size(0),max_len))
      ## |length| = (beam_size*batch_size,)
      ## |idx| = (beam_size*batch_size, max_len,)
      
      expanded_length = length.unsqueeze(1).expand(idx.size())
      ## |expanded_length| = (beam_size*batch_size, max_len)
      
      mask = idx >= expanded_length
      ## |mask| = (beam_size*batch_size, max_len)

    return mask


  ## after all time steps
  ## not in forward  
  def calc_path_score(self, length, attn_weights) :
    ## |length| = (beam_size * bathc_size, 1, 1)
    ## |attn_weights| = (beam_size *  batch_size, enc_length, dec_length)

    length_pen = self.length_penalty(length)
    ## |length_pen| = (beam_size * batch_size, 1, 1)

    conver_pen = self.convergence_penalty(attn_weights)
    ## |conver_pen| = (beam_size * batch_size, 1, 1)

    path_score = (torch.bmm(self.path_probs, length_pen) + conver_pen).contiguous() 
    ## |path_score| = (beam_size * batch_size, 1, 1)

    return path_score 

  ## after calc_path_score
  ## not in forward()
  def get_best_path(self, path_score, mask) :
    ## |path_score| = (beam_size * batch_size, 1, 1)
    ## |instance_len| = (beam_size*batch_size, 1, 1)

    path_score = self.merge_eqaul_instance(path_score)
    ## |path_score| = |instance_len| = (batch_size, 1, beam_size)

    path = torch.cat(self.path, dim=1)
    ## |path| = (beam_size*batch_size, length, 1)

    max_path_id = torch.argmax(path_score, dim=-1).squeeze()
    ## |max_path_id| = (batch_size,)
    
    max_path_id = torch.cat([ i%self.batch_size + path_id*self.batch_size for i, path_id in enumerate(max_path_id)]).squeeze()
    ## |max_path_id| = (batch_size,)

    best_path = path[max_path_id].contiguous()
    ## |best_path| = (batch_size, length, )

    best_path = best_path.masked_fill_(mask, -1)
    ## |best_path| = (batch_size, length)

    return best_path

In [15]:
class TranslationModel(nn.Module) :
  def __init__(self, input_size, output_size, emb_size, hidden_size, beam_size, num_layer, bidirectional, drop_out, lp_alpha, lp_length, cp_beta, cp_min_val, max_len) :
    super(TranslationModel, self).__init__()
    
    self.num_layer = num_layer
    self.num_direction = 2 if bidirectional else 1
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.max_len = max_len


    self.src_emb = nn.Embedding(input_size, emb_size)
    self.tgt_emb = nn.Embedding(output_size, emb_size)
    self.enc = LSTMEncoder(emb_size, hidden_size, num_layer=num_layer, bidirectional=bidirectional, dropout=drop_out)
    self.dec = LSTMDecoder(emb_size, hidden_size, num_layer = num_layer, dropout=drop_out)
    self.attn = Attention(hidden_size)
    self.concat = nn.Linear(2*hidden_size, hidden_size)
    self.tanh = nn.Tanh()
    
    self.gen = Generator(hidden_size, output_size)
    
    self.beam_search = BeamSearch(beam_size, lp_alpha, lp_length, cp_beta, cp_min_val)


  def forward(self, src, tgt) :
        ## 1. src = (src_data, src_length)의 튜플인 경우 
        ##   |src_data| = (batch_size, src_length,)
        ##   |src_length| = (batch_size, )
        ## 2. src = src_data 
        ##   |src_data| = (batch_size, src_length, )

        ## |tgt| = (batch_size, tgt_length, )

    batch_size = src[0].size(0)
    mask = None
    src_length = None
    if isinstance(src, tuple) :
      src_data, src_length = src
      mask = self.make_mask(src_data, src_length)
      ## |mask| = (batch_size, length)
    else :
      src_data = src

    if isinstance(tgt, tuple) : 
      tgt = tgt[0]

    src_emb = self.src_emb(src_data)
    ## |src_emb| = (batch_size, length, emb_size)

    enc_hs, init_dec_hc = self.enc((src_emb,src_length))
    init_dec_h, init_dec_c = init_dec_hc 
    ## |enc_hs| = (batch_size, length, hidden_size) -> 후에 attention을 위해 사용 
    ## |init_dec_h| = (2*num_layer,batch_size,hidden_size//2) -> decoder의 각 layer의 hidden state의 초기 값으로 사용 
    ## |init_dec_c| = (2*num_layer,batch_size,hidden_size//2) -> decoder의 각 layer의 cell state의 초기 값으로 사용 

    init_dec_h = init_dec_h.transpose(0,1).contiguous().view(batch_size, -1, self.hidden_size).transpose(0,1).contiguous()
    init_dec_c = init_dec_c.transpose(0,1).contiguous().view(batch_size, -1, self.hidden_size).transpose(0,1).contiguous()
    ## |init_dec_h| = (num_layer, batch_size, hidden_size)
    ## |init_dec_c| = (num_layer, batch_size, hidden_size) 


    tgt_emb = self.tgt_emb(tgt)
    outputs = list()
    
    inp_feed = None
    dec_hc = (init_dec_h,init_dec_c)
    
    for i in range(tgt.size(1)) :
      ## Embedding 
      cur_emb = tgt_emb[:,i,:].unsqueeze(1)
      ## |cur_emb| = (batch_size, emb_size)


      ## Decoder 
      dec_out, dec_hc = self.dec(cur_emb, inp_feed, dec_hc)
      ## |dec_out| = (batch_size, 1, hidden_size)
      ## |dec_hc|
      ##   |dec_hc[0]| = (num_layer, batch_size, hidden_size)
      ##   |dec_hc[1]| = (num_layer, batch_size, hidden_size)
      
      ## Attention 
      context_vector, _ = self.attn(enc_hs, dec_out, mask)
      ## |context_vector| = (batch_size, 1, hidden_size)

      inp_feed = self.tanh(self.concat(torch.cat([dec_out, context_vector], dim=-1))) 
      ## |inp_feed| = (batch_size, 1, hidden_size)

      outputs.append(inp_feed)
    
    y_hat = torch.cat(outputs, dim=1)
    y_hat = self.gen(y_hat)
    ## |y_hat| = (batch_size, length, output_size)

    return y_hat

  def beam_search(self, src, bos_id, eos_id) :
    ## 1. src = (src_data, src_length)의 튜플인 경우 
    ##   |src_data| = (batch_size, src_length, input_size)
    ##   |src_length| = (batch_size, )
    ## 2. src = src_data 
    ##   |src_data| = (batch_size, src_length, input_size)

    ## 3. bos_id = (1,)

    batch_size = src.size(0)
    if isinstance(src, tuple) :
      ## emb_vec_data와 emb_vec_length는 mini batch 내에서 pad를 제외한 토큰의 개수를 기준으로
      ## 내림차순으로 정렬되어 있음. 이는 효율성 뿐만 아니라 pad를 계산하지 않아 정확도를 높이
      ## 기 위한 목적도 존재 
      ## 출처 : https://simonjisu.github.io/nlp/2018/07/05/packedsequence.html
      src_data, src_length = src
      #src_data = pack_padded_sequence(src_data, src_length)
    else :
      src_data = src

    src_emb = self.src_emb(src_data)
    ## |src_emb| = (batch_size, length, emb_size)

    enc_hs, init_dec_hc = self.enc((src_emb,src_length)) 
    ## |enc_hs| = (batch_size, length, hidden_size) -> 후에 attention을 위해 사용 
    ## |init_dec_hc[0]| = (2*num_layer,batch_size,hidden_size//2) -> decoder의 각 layer의 hidden state의 초기 값으로 사용 
    ## |init_dec_hc[1]| = (2*num_layer,batch_size,hidden_size//2) -> decoder의 각 layer의 cell state의 초기 값으로 사용 

    init_dec_h = init_dec_hc[0].transpose(0,1).contiguous().view(batch_size, -1, self.hidden_size).transpose(0,1).contiguous()
    init_dec_c = init_dec_hc[1].transpose(0,1).contiguous().view(batch_size, -1, self.hidden_size).transpose(0,1).contiguous()
    ## |init_dec_h| = (num_layer, batch_size, hidden_size)
    ## |init_dec_c| = (num_layer, batch_size, hidden_size) 

    inp = torch.tensor([ bos_id for i in range(batch_size)])
    dec_hc = (init_dec_h,init_dec_c)
    mask = None 
    end_flag = torch.tensor([False for i in range(batch_size)])
    attn_weights = []
    for i in range(self.max_len) :
      ## Embedding 
      tgt_emb = self.tgt_emb(inp).unsqueeze(1)
      ## |tgt_emb| = (batch_size, 1, emb_size) or (beam_size*batch_size, 1, emb_size)

      ## Decoder 
      dec_out, dec_hc = self.dec(tgt_emb, dec_hc)
      ## |dec_out| = (batch_size, 1, hidden_size) or (beam_size * batch_size, 1 hidden_size)
      ## |dec_hc| = (num_layer, batch_size, hidden_size) or (num_layer, beam_size * batch_size, hidden_size)
      
      ## Attention 
      context_vector, attn_weight = self.attn(enc_hs, dec_out)
      ## |context_vector| = (batch_size, 1, hidden_size) or (beam_size * batch_size, 1, hidden_size)

      ctx_refl_dec = self.tanh(self.concat(torch.cat([dec_out, context_vector], dim=-1))) 
      ## |ctx_refl_dec| = (batch_size, 1, hidden_size) or (beam_size * batch_size, 1, hidden_size)

      y_hat = self.gen(ctx_refl_dec)
      ## |y_hat| = (batch_size, 1, output_size) or (beam_size * batch_size, 1, output_size)


      ## BeamSearch 
      inp, dec_hc = self.beam_search(y_hat, dec_hc, ctx_refl_dec, mask)
      ## |inp| = (beam_size*batch_size, 1, 1)
      ## |dec_hc| 
      ##  |dec_h| = (num_layers, beam_size*batch_size, hidden_size)
      ##  |dec_c| = (num_layers, beam_size*batch_size, hidden_size)

      ## 종료된 부분을 나타내는 end flag 계산 
      end_flag = (end_flag) | (inp == eos_id)
      ## |end_flag| = (batch_size, ) or (beam_size * batch_size, ) 
      if end_flag.size(0) == batch_size :
        end_flag = end_flag.exapnd(self.beam_size*batch_size, )
        ## |end_flag| = (beam_size * batchs_size, )
        
      ## 종료된 부분의 길이 저장 
      length[(inp == eos_id)] = i

      ## 현재 timestep의 attention weight 저장 
      attn_weights.append(attn_weight)

      ## 모든 Instance에 대하여 추론이 끝난 경우 종료 
      if torch.sum(end_flag) == self.beam_size*batch_size : 
        break

      ## 종료된 부분을 표시하는 mask 구하기
      mask = self.beam_search.make_mask(end_flag, mask)
      ## |mask| = (bathc_size, ) or (beam_size*batch_size, )

    ## 최종 path 선택 
    attn_weights = torch.cat(attn_weights, dim=1).transpose(0,1).contiguous()
    ## |attn_weights| = (batch_size*beam_size, enc_length, dec_length)
    path_score = beam_search.calc_path_score(length)
    ## |path_score| = (batch_size*beam_size,)
    transl_result = get_best_path(path_score, mask)
    ## |tranl_result| = (batch_size, )
    
    return transl_result 
  
  def make_mask(self, src, src_length) : 
    mask = list()

    max_length = max(src_length)

    for cur_length in src_length : 
      if max_length - cur_length > 0 : 
        mask.append(torch.cat([src.new_ones(1,cur_length).zero_(), src.new_ones(1,max_length-cur_length)], dim=-1))
      else : 
        mask.append(src.new_ones(1, cur_length).zero_())
    
    mask = torch.cat(mask, dim=0).bool()

    return mask


In [23]:
tl_model = TranslationModel(input_size, output_size, embed_size, hidden_size, beam_size, num_layer, enc_bidirectional, drop_out, lp_alpha, lp_length, cp_beta, cp_min_val, max_len)
tl_model.to(device)

TranslationModel(
  (src_emb): Embedding(24393, 512)
  (tgt_emb): Embedding(44380, 512)
  (enc): LSTMEncoder(
    (encoder): LSTM(512, 384, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (dec): LSTMDecoder(
    (decoder): LSTM(1280, 768, num_layers=4, batch_first=True, dropout=0.2)
  )
  (attn): Attention(
    (make_query): Linear(in_features=768, out_features=768, bias=False)
    (softmax): Softmax(dim=-1)
  )
  (concat): Linear(in_features=1536, out_features=768, bias=True)
  (tanh): Tanh()
  (gen): Generator(
    (transform): Linear(in_features=768, out_features=44380, bias=True)
    (softmax): LogSoftmax(dim=-1)
  )
  (beam_search): BeamSearch()
)

In [24]:
## LogSoftmax + NLLLoss 
loss_weight = torch.ones(tl_model.output_size)
loss_weight[train_dl.PAD_ID] = 0.
crit = nn.NLLLoss(loss_weight, reduction='sum').to(device)
optimizer = optim.Adam(tl_model.parameters(), lr = learning_rate, )

In [25]:
## for amp 
scaler = GradScaler()

In [26]:
## to save 
## Hyper Parameters 
hyper_params = {'train_src_fn' : train_src_fn, 'train_tgt_fn' : train_tgt_fn, 'valid_src_fn' : valid_src_fn,
                'valid_tgt_fn' : valid_tgt_fn, 'max_len' : max_len, 'init_token' : init_token,
                'eos_token' : eos_token, 'batch_size' : batch_size, 'device' : device, 'max_vocab' : max_vocab,
                'num_epochs' : num_epochs, 'train_src_vocab' : train_dl.src.vocab, 'train_tgt_vocab':train_dl.tgt.vocab , 'valid_src_vocab' : valid_dl.src.vocab, 'valid_tgt_vocab' : valid_dl.tgt.vocab,
                'input_size' : input_size, 'output_size' : output_size, 'embed_size' : embed_size, 'hidden_size' : hidden_size,
                'num_layer' : num_layer, 'enc_bidirectional' : enc_bidirectional, 'drop_out' : drop_out, 'max_clip_norm': max_clip_norm, 'beam_size' : beam_size,
                'lp_alpha' : lp_alpha, 'lp_length' : lp_length, 'cp_beta' : cp_beta, 'cp_min_val' : cp_min_val, 'learning_rate' : learning_rate,
                'grad_update_period' : grad_update_period
                }



In [ ]:
#loss/ ppl 
train_loss_sum = 0
train_ppl_sum = 0

valid_loss_sum = 0
valid_ppl_sum = 0

train_loss_list = list()
train_ppl_list = list() 

valid_loss_list = list()
valid_ppl_list = list()

train_loss_sum_list = list() 
train_ppl_sum_list = list()
valid_loss_sum_list = list()
valid_ppl_sum_list = list()

for cur_epoch in range(num_epochs) :
  tl_model.train()

  train_loss_sum = 0.; train_ppl_sum = 0.; valid_loss_sum = 0.; valid_ppl_sum =0.
  for i, train_batch in enumerate(train_dl.data_iter) :
    src = train_batch.src; tgt = train_batch.tgt
    ## |src|
    ##  |src[0]| = (batch_size, length, )
    ##  |src[1]| = (batch_size) 
    ## |tgt|
    ##  |tgt[0]| = (batch_size, length, ) 
    ##  |tgt[1]| = (batch_size)

    ## this is for gpu memory leakage 
    src = (src[0].to(device), src[1]);
    tgt = (tgt[0].to(device), tgt[1]);

    
    with autocast() :
      ## FeedForward 
      log_probs = tl_model(src, tgt[0][:,:-1])  ## extract <EOS> from target 
      ## |log_probs| = (batch_size, length, output_size)

      log_probs = log_probs.view(-1, log_probs.size(-1))
      ## |log_probs| = (batch_size * length, output_size)

      loss = crit(log_probs, tgt[0][:,1:].contiguous().view(-1))
      loss_for_gradient = loss / (src[0].size(0) * grad_update_period)
    
    ## Scale loss. Calls backward() on scaled loss to create scaled gradients. 
    ## Backward passes under autocast are not recommended. 
    ## Backward ops run in the same dtype autocast chose for corresponding forward ops.
    ## Accumulates scaled gradients 
    scaler.scale(loss_for_gradient).backward()

    ## update for every periods
    if (i+1) % grad_update_period == 0 :
      ## Gradient Clipping because Gradient Explood or Vanish in RNN based Models 
      tc_utils.clip_grad_norm_(tl_model.parameters(), max_clip_norm)

      ## scaler.step() first unscale the gradients of the optimizer's assigned params. 
      ## If these gradients do not contain infs or NaNs, optimizer.step() is then called, 
      ## otherwise optimizer.step() is skipped 
      scaler.step(optimizer)

      ## Gradient Descent

      ## Updates the scale for next iteration. 
      scaler.update()  

      ## Clear Gradient for next period 
      optimizer.zero_grad()

    ## Calculate loss and perplexity 

    all_words_num = int(train_batch.tgt[1].sum())
    ## tgt[1] is length of each instance 

    ## loss mean for each word in mini batch 
    loss = float(loss / all_words_num)
    ppl = np.exp(loss)
    
    train_loss_list.append(loss)
    train_ppl_list.append(ppl)
    print_st = max(0, len(train_loss_list) - 100)
    print(f'Train Epoch #{cur_epoch+1}, Batch {i+1}/{len(train_dl.data_iter)} loss = {sum(train_loss_list[print_st:])/len(train_loss_list[print_st:])}, ppl = {sum(train_ppl_list[print_st:])/len(train_ppl_list[print_st:])}')  
    
  train_loss_sum_list.append(train_loss_sum)
  train_ppl_sum_list.append(train_ppl_sum)

  tl_model.eval()
  for i, valid_batch in enumerate(valid_dl.data_iter) :
    src = valid_batch.src;  tgt = valid_batch.tgt
    ## |src| 
    ##   |src[0]| = (batch_size, length, )
    ##   |src[1]| = (batch_size, )
    ## |tgt| 
    ##   |tgt[0]| = (batch_size, length, )
    ##   |tgt[1]| = (batch_size,)

    with torch.no_grad() :
      src = (src[0].to(device), src[1])
      tgt = (tgt[0].to(device), tgt[1])
      ## |src[0]| = (length, batch_size)
      ## |tgt[0]| = (length, batch_size)
      
      with autocast() : 
      ## FeedForward 
       log_probs = tl_model(src, tgt[0][:,:-1]) ## extract <EOS> from target 
       ## |log_probs| = (batch_size * length, output_size)
       
       log_probs = log_probs.view(-1, log_probs.size(-1))
       ## |log_probs| = (batch_size * length, output_size)
       
       loss = crit(log_probs, tgt[0][:,1:].contiguous().view(-1)) 
       loss_for_gradient = loss / (src[0].size(0) * grad_update_period)
    
    
    ## Calculate loss and perplexity 
    ## loss mean for each word in mini batch  
    all_words_num = int(tgt[1].sum())
    loss = float(loss/all_words_num)
    ppl = np.exp(loss) 
    
    valid_loss_list.append(float(loss))
    valid_ppl_list.append(float(ppl))
    
    print_st = max(0, len(valid_loss_list) - 100)
    print(f'Valid Epoch #{cur_epoch+1}, Batch {i+1}/{len(valid_dl.data_iter)} loss = {sum(valid_loss_list[print_st:])/len(valid_loss_list[print_st:])}, ppl = {sum(valid_ppl_list[print_st:])/len(valid_ppl_list[print_st:])}')

    valid_loss_sum += loss
    valid_ppl_sum += ppl
    
  valid_loss_sum_list.append(valid_loss_sum)
  valid_ppl_sum_list.append(valid_ppl_sum)

  ## Save Model for each iteration 
  SAVE_PATH = f'./model_checkpoints/cur_epoch_{cur_epoch+1}_train_loss_{int(train_loss_sum/len(train_dl.data_iter))}_train_ppl_{int(train_ppl_sum/len(train_dl.data_iter))}_valid_loss_{int(valid_loss_sum/len(valid_dl.data_iter))}_valid_ppl_{int(valid_ppl_sum/len(valid_dl.data_iter))}.pt'
  torch.save({'hyper_params' : hyper_params, 'train_loss_sum' : train_loss_sum, 'train_ppl_sum' : train_ppl_sum, 'valid_loss_sum' : valid_loss_sum, 'valid_ppl_sum' : valid_ppl_sum,
              'model_state_dict' : tl_model.state_dict(), 'optim_state_dict' : optimizer.state_dict(), 'grad_scaler_state_dict' : scaler.state_dict()}, SAVE_PATH)


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Valid Epoch #7, Batch 342/4160 loss = 1.57185626745224, ppl = 4.860639045796545
Valid Epoch #7, Batch 343/4160 loss = 1.5709064674377442, ppl = 4.855587284433728
Valid Epoch #7, Batch 344/4160 loss = 1.5721943867206574, ppl = 4.86247705281879
Valid Epoch #7, Batch 345/4160 loss = 1.573664404153824, ppl = 4.871388917215114
Valid Epoch #7, Batch 346/4160 loss = 1.5747041153907775, ppl = 4.877385068528685
Valid Epoch #7, Batch 347/4160 loss = 1.574900517463684, ppl = 4.878384243009688
Valid Epoch #7, Batch 348/4160 loss = 1.5767981803417206, ppl = 4.887215936574237
Valid Epoch #7, Batch 349/4160 loss = 1.5771223533153533, ppl = 4.888794966848448
Valid Epoch #7, Batch 350/4160 loss = 1.5778649163246155, ppl = 4.892972524273978
Valid Epoch #7, Batch 351/4160 loss = 1.5789142739772797, ppl = 4.897959272907871
Valid Epoch #7, Batch 352/4160 loss = 1.5789950501918792, ppl = 4.898376369099401
Valid Epoch #7, Batch 353/4160 loss = 1.577855154275894, ppl = 4.89

In [17]:
  ## load_model 
SAVE_PATH = './model_checkpoints/cur_epoch_11_train_loss_0_train_ppl_0_valid_loss_1_valid_ppl_4.pt'
checkpoint = torch.load(SAVE_PATH)

## Hyper Parameter for Data 
train_src_fn = checkpoint['hyper_params']['train_src_fn']
train_tgt_fn = checkpoint['hyper_params']['train_tgt_fn']
valid_src_fn = checkpoint['hyper_params']['valid_src_fn']
valid_tgt_fn = checkpoint['hyper_params']['valid_tgt_fn']
max_len = checkpoint['hyper_params']['max_len']
init_token = checkpoint['hyper_params']['init_token']
eos_token= checkpoint['hyper_params']['eos_token']
batch_size= checkpoint['hyper_params']['batch_size']
device = checkpoint['hyper_params']['device']
max_vocab = checkpoint['hyper_params']['max_vocab']

num_epochs =checkpoint['hyper_params']['num_epochs']

train_src_vocab = checkpoint['hyper_params']['train_src_vocab']
train_tgt_vocab = checkpoint['hyper_params']['train_tgt_vocab']

input_size = checkpoint['hyper_params']['input_size']
output_size = checkpoint['hyper_params']['output_size']


embed_size = checkpoint['hyper_params']['embed_size']
hidden_size = checkpoint['hyper_params']['hidden_size']
num_layer = checkpoint['hyper_params']['num_layer']
enc_bidirectional = checkpoint['hyper_params']['enc_bidirectional']
drop_out = checkpoint['hyper_params']['drop_out']
max_clip_norm = checkpoint['hyper_params']['max_clip_norm']


beam_size = checkpoint['hyper_params']['beam_size']
lp_alpha = checkpoint['hyper_params']['lp_alpha']
lp_length = checkpoint['hyper_params']['lp_length']
cp_beta = checkpoint['hyper_params']['cp_beta']
cp_min_val = checkpoint['hyper_params']['cp_min_val']

learning_rate = checkpoint['hyper_params']['learning_rate']
grad_update_period = checkpoint['hyper_params']['grad_update_period'] 

##metric 
train_loss_sum = checkpoint['train_loss_sum']
train_ppl_sum = checkpoint['train_ppl_sum']

valid_loss_sum = checkpoint['valid_loss_sum']
valid_ppl_sum = checkpoint['valid_ppl_sum']

In [18]:
train_dl =  DataLoader(train_src_fn, train_tgt_fn, max_len, init_token, eos_token, batch_size, -1, max_vocab)   ## device = -1 because gpu memory leakage -> batchwise loading 
test_valid_dl =  DataLoader(valid_src_fn, valid_tgt_fn, max_len, init_token, eos_token, batch_size, -1, max_vocab)   ## device = -1 because gpu memory leakage -> batchwise loading 

train_dl.set_vocab_for_reproduction(train_src_vocab, train_tgt_vocab)
test_valid_dl.set_vocab_for_reproduction(train_src_vocab, train_tgt_vocab)

tl_model = TranslationModel(input_size, output_size, embed_size, hidden_size, beam_size, num_layer, enc_bidirectional, drop_out, lp_alpha, lp_length, cp_beta, cp_min_val, max_len)
tl_model.load_state_dict(checkpoint['model_state_dict'])
tl_model.to(device)

## LogSoftmax + NLLLoss 
loss_weight = torch.ones(tl_model.output_size)
loss_weight[train_dl.PAD_ID] = 0.
crit = nn.NLLLoss(loss_weight, reduction='sum').to(device)
optimizer = optim.Adam(tl_model.parameters(), lr = learning_rate, )
optimizer.load_state_dict(checkpoint['optim_state_dict'])

## for amp 
scaler = GradScaler()

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information

In [20]:
## to save 
## Hyper Parameters 
hyper_params = {'train_src_fn' : train_src_fn, 'train_tgt_fn' : train_tgt_fn, 'valid_src_fn' : valid_src_fn,
                'valid_tgt_fn' : valid_tgt_fn, 'max_len' : max_len, 'init_token' : init_token,
                'eos_token' : eos_token, 'batch_size' : batch_size, 'device' : device, 'max_vocab' : max_vocab,
                'num_epochs' : num_epochs, 'train_src_vocab' : train_dl.src.vocab, 'train_tgt_vocab':train_dl.tgt.vocab , 
                'input_size' : input_size, 'output_size' : output_size, 'embed_size' : embed_size, 'hidden_size' : hidden_size,
                'num_layer' : num_layer, 'enc_bidirectional' : enc_bidirectional, 'drop_out' : drop_out, 'max_clip_norm': max_clip_norm, 'beam_size' : beam_size,
                'lp_alpha' : lp_alpha, 'lp_length' : lp_length, 'cp_beta' : cp_beta, 'cp_min_val' : cp_min_val, 'learning_rate' : learning_rate,
                'grad_update_period' : grad_update_period
                }



In [ ]:
#loss/ ppl 
train_loss_sum = 0
train_ppl_sum = 0

valid_loss_sum = 0
valid_ppl_sum = 0

train_loss_list = list()
train_ppl_list = list() 

valid_loss_list = list()
valid_ppl_list = list()

train_loss_sum_list = list() 
train_ppl_sum_list = list()
valid_loss_sum_list = list()
valid_ppl_sum_list = list()

for cur_epoch in range(12,num_epochs) :
  tl_model.train()

  train_loss_sum = 0.; train_ppl_sum = 0.; valid_loss_sum = 0.; valid_ppl_sum =0.
  for i, train_batch in enumerate(train_dl.data_iter) :
    src = train_batch.src; tgt = train_batch.tgt
    ## |src|
    ##  |src[0]| = (batch_size, length, )
    ##  |src[1]| = (batch_size) 
    ## |tgt|
    ##  |tgt[0]| = (batch_size, length, ) 
    ##  |tgt[1]| = (batch_size)

    ## this is for gpu memory leakage 
    src = (src[0].to(device), src[1]);
    tgt = (tgt[0].to(device), tgt[1]);

    
    with autocast() :
      ## FeedForward 
      log_probs = tl_model(src, tgt[0][:,:-1])  ## extract <EOS> from target 
      ## |log_probs| = (batch_size, length, output_size)

      log_probs = log_probs.view(-1, log_probs.size(-1))
      ## |log_probs| = (batch_size * length, output_size)

      loss = crit(log_probs, tgt[0][:,1:].contiguous().view(-1))
      loss_for_gradient = loss / (src[0].size(0) * grad_update_period)
    
    ## Scale loss. Calls backward() on scaled loss to create scaled gradients. 
    ## Backward passes under autocast are not recommended. 
    ## Backward ops run in the same dtype autocast chose for corresponding forward ops.
    ## Accumulates scaled gradients 
    scaler.scale(loss_for_gradient).backward()

    ## update for every periods
    if (i+1) % grad_update_period == 0 :
      ## Gradient Clipping because Gradient Explood or Vanish in RNN based Models 
      tc_utils.clip_grad_norm_(tl_model.parameters(), max_clip_norm)

      ## scaler.step() first unscale the gradients of the optimizer's assigned params. 
      ## If these gradients do not contain infs or NaNs, optimizer.step() is then called, 
      ## otherwise optimizer.step() is skipped 
      scaler.step(optimizer)

      ## Gradient Descent

      ## Updates the scale for next iteration. 
      scaler.update()  

      ## Clear Gradient for next period 
      optimizer.zero_grad()

    ## Calculate loss and perplexity 

    all_words_num = int(train_batch.tgt[1].sum())
    ## tgt[1] is length of each instance 

    ## loss mean for each word in mini batch 
    loss = float(loss / all_words_num)
    ppl = np.exp(loss)
    
    train_loss_list.append(loss)
    train_ppl_list.append(ppl)
    print_st = max(0, len(train_loss_list) - 100)
    print(f'Train Epoch #{cur_epoch+1}, Batch {i+1}/{len(train_dl.data_iter)} loss = {sum(train_loss_list[print_st:])/len(train_loss_list[print_st:])}, ppl = {sum(train_ppl_list[print_st:])/len(train_ppl_list[print_st:])}')  
    
  train_loss_sum_list.append(train_loss_sum)
  train_ppl_sum_list.append(train_ppl_sum)

  tl_model.eval()
  for i, valid_batch in enumerate(test_valid_dl.data_iter) :
    src = valid_batch.src;  tgt = valid_batch.tgt
    ## |src| 
    ##   |src[0]| = (batch_size, length, )
    ##   |src[1]| = (batch_size, )
    ## |tgt| 
    ##   |tgt[0]| = (batch_size, length, )
    ##   |tgt[1]| = (batch_size,)

    with torch.no_grad() :
      src = (src[0].to(device), src[1])
      tgt = (tgt[0].to(device), tgt[1])
      ## |src[0]| = (length, batch_size)
      ## |tgt[0]| = (length, batch_size)
      
      with autocast() : 
      ## FeedForward 
       log_probs = tl_model(src, tgt[0][:,:-1]) ## extract <EOS> from target 
       ## |log_probs| = (batch_size * length, output_size)
       
       log_probs = log_probs.view(-1, log_probs.size(-1))
       ## |log_probs| = (batch_size * length, output_size)
       
       loss = crit(log_probs, tgt[0][:,1:].contiguous().view(-1)) 
       loss_for_gradient = loss / (src[0].size(0) * grad_update_period)
    
    
    ## Calculate loss and perplexity 
    ## loss mean for each word in mini batch  
    all_words_num = int(tgt[1].sum())
    loss = float(loss/all_words_num)
    ppl = np.exp(loss) 
    
    valid_loss_list.append(float(loss))
    valid_ppl_list.append(float(ppl))
    
    print_st = max(0, len(valid_loss_list) - 100)
    print(f'Valid Epoch #{cur_epoch+1}, Batch {i+1}/{len(test_valid_dl.data_iter)} loss = {sum(valid_loss_list[print_st:])/len(valid_loss_list[print_st:])}, ppl = {sum(valid_ppl_list[print_st:])/len(valid_ppl_list[print_st:])}')

    valid_loss_sum += loss
    valid_ppl_sum += ppl
    
  valid_loss_sum_list.append(valid_loss_sum)
  valid_ppl_sum_list.append(valid_ppl_sum)

  ## Save Model for each iteration 
  SAVE_PATH = f'./model_checkpoints/cur_epoch_{cur_epoch+1}_train_loss_{int(train_loss_sum/len(train_dl.data_iter))}_train_ppl_{int(train_ppl_sum/len(train_dl.data_iter))}_valid_loss_{int(valid_loss_sum/len(test_valid_dl.data_iter))}_valid_ppl_{int(valid_ppl_sum/len(test_valid_dl.data_iter))}.pt'
  torch.save({'hyper_params' : hyper_params, 'train_loss_sum' : train_loss_sum, 'train_ppl_sum' : train_ppl_sum, 'valid_loss_sum' : valid_loss_sum, 'valid_ppl_sum' : valid_ppl_sum,
              'model_state_dict' : tl_model.state_dict(), 'optim_state_dict' : optimizer.state_dict(), 'grad_scaler_state_dict' : scaler.state_dict()}, SAVE_PATH)


/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Train Epoch #17, Batch 3999/24958 loss = 1.2438573920726776, ppl = 3.4923436927872324
Train Epoch #17, Batch 4000/24958 loss = 1.2436955642700196, ppl = 3.491793963830811
Train Epoch #17, Batch 4001/24958 loss = 1.2427750408649445, ppl = 3.48882126261266
Train Epoch #17, Batch 4002/24958 loss = 1.242962054014206, ppl = 3.48950904361222
Train Epoch #17, Batch 4003/24958 loss = 1.2417497885227204, ppl = 3.485440927672621
Train Epoch #17, Batch 4004/24958 loss = 1.2412441551685334, ppl = 3.4837914917827857
Train Epoch #17, Batch 4005/24958 loss = 1.2417908883094788, ppl = 3.4856116075581793
Train Epoch #17, Batch 4006/24958 loss = 1.2416067230701446, ppl = 3.4849972720913365
Train Epoch #17, Batch 4007/24958 loss = 1.240616512298584, ppl = 3.481924121856646
Train Epoch #17, Batch 4008/24958 loss = 1.2396324396133422, ppl = 3.4783748275531257
Train Epoch #17, Batch 4009/24958 loss = 1.2388035178184509, ppl = 3.4750870103915177
Train Epoch #17, Batch 4010

In [21]:
  SAVE_PATH = f'./model_checkpoints/cur_epoch_{cur_epoch+1}_train_loss_{int(train_loss_sum/len(train_dl.data_iter))}_train_ppl_{int(train_ppl_sum/len(train_dl.data_iter))}_valid_loss_{int(valid_loss_sum/len(test_valid_dl.data_iter))}_valid_ppl_{int(valid_ppl_sum/len(test_valid_dl.data_iter))}.pt'
  torch.save({'hyper_params' : hyper_params, 'train_loss_sum' : train_loss_sum, 'train_ppl_sum' : train_ppl_sum, 'valid_loss_sum' : valid_loss_sum, 'valid_ppl_sum' : valid_ppl_sum,
              'model_state_dict' : tl_model.state_dict(), 'optim_state_dict' : optimizer.state_dict(), 'grad_scaler_state_dict' : scaler.state_dict()}, SAVE_PATH)